# Learning Word Embeddings

```
Second Year Project 2020 ~ Lab week 4, part I ~ B.Plank
```

So far you learned about distributional semantics (vector semantics) in both the traditional and modern neural way, and you qualitatively worked with pre-trained (off-the-shelf) word embeddings in the last assignment.

In this assignment, you will learn how to implement a neural network  to learn word embeddings, namely the *Continous Bag of Words* (CBOW) model for word embeddings:

- You will learn how to preprocess the data for this task, from the raw sentences in the corpus to the required input for this task. You will also learn how to implement the model in Keras, how to train it and how to ultimately obtain word embeddings from it.

- You will implement CBOW for learning word embeddings, to learn and understand the core idea behind learning neural word representations, while getting aquainted with deep learning in Keras.

### "CBOW, CBOW, CBOW.. What?! Which?!"



CBOW is a model proposed by [Mikolov et al., 2013](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf).

It is a simple neural method to learn word embeddings and it is one of the two core algorithms in the `word2vec` toolkit (see figure below). Note that, besides its usage here to learn word embeddings, CBOW is also a more general term used to refer to any input representation which consists of (some) way of aggregating a set of word embeddings. Hence its name, the continous BOW representation. You can in fact use such a similar representation (e.g., the average of the embeddings of words) for other tasks as well, such as text classification. Here, CBOW is meant in its original formulation: a network over the *sum* of embeddings of context words aimed at predicting the middle target word. It is related in spirit to a language model, but instead framed as a classification task (with context available on both size) and hence bears more similarities with a *[word close test](https://en.wikipedia.org/wiki/Cloze_test)*.

Illustration of the CBOW model (in comparison to the skip-gram):
<img src="pics/cbow-vs-skipgram.png">





##  <font color='blue'>Task 1</font>: Representing the data

Given a corpus, extract the training data for the CBOW model using a window size of 2 words on each side of the target word.

* learn how to extract the data for this task and represent it in Keras
* learn how to transform your labels into the n-hot representations used in Keras

<img src="pics/cbow-context-targets.png">

Hints:
* remember to `"<pad>"` the input when the window size is smaller than the expected window size; reserve the first (0) index for this special token
* once you extracted all windows and target pairs, convert them to the expected data format, as illustrated in the example below, i.e., each word (or the `"<pad>"` token) is represented by an index (using the `defaultdict` might be helpful to store a `word2idx` mapping)
* convert the labels (target words) into Keras multi-class representations - hint: use `keras.utils.to_categorical(y, num_classes=None, dtype='float32')`

Example:

Given the following tiny corpus:
```
tiny_corpus = ["this is an example", "another example", "I love deep learning"]
```

To create the `train_X` data, you first need to extract n-gram windows and the target words:

```
label,context
this ['<pad>', '<pad>', 'is', 'an']
is ['<pad>', 'this', 'an', 'example']
an ['this', 'is', 'example', '<pad>']
...
```

And convert them into numeric format, where each word token is represented by its unique index:

```
[[ 0  0  1  2]
 [ 0  3  2  4]
 [ 3  1  4  0]
 [ 1  2  0  0]
 [ 0  0  4  5]
 [ 0  6  5  0]
 [ 6  4  0  0]
 [ 0  0  7  8]
 [ 0  9  8 10]
 [ 9  7 10  0]
 [ 7  8  0  0]]
```

Similarly, to create `y_train`, you need to extract the labels (target tokens), convert each (token) into an index to finally transform it into a *one-hot vector* for each label (hint: using `keras.utils.to_categorical`).

In [2]:
tiny_corpus = ["this is an example", "another example", "I love deep learning"]

Suggestion: Implement all your steps first on the `tiny_corpus` data. Then test your implementation on the provided data `sample.txt`.

In [29]:
from collections import defaultdict 
from keras import utils
import numpy as np
import re
from tensorflow import keras

## global settings
PAD = "<pad>"
window_size=2

### your code here
word2idx = defaultdict(int) # a vocabulary mapper: word to index, with <pad>
word2idx["<pad>"] = 0 # reserve 0 for padding

# expected output
train_X = None # a matrix of instances x context windows (numeric)
train_y = None # the target labels, encoded in Keras to_categorical




In [36]:
train_x_list = list()
train_y_list = list()

for line in tiny_corpus:

    # tokenize and add padding
    token = re.compile('\w+[\'-]\w+|Mrs?\.|[\w\']+')
    tokens = token.findall(line)
    for _ in range(window_size):
        tokens.insert(0, PAD)
        tokens.append(PAD)

    # build training data
    for idx, w in enumerate(tokens[window_size:-window_size]):
        #print('w = {}'.format(w))
        cl = list()
        idx += window_size
        for b_n in range(window_size, 0, -1):
            my_neighbor = tokens[idx - b_n]
            if my_neighbor not in word2idx:
                word2idx[my_neighbor] = len(word2idx)
            cl.append(word2idx[my_neighbor])
        for a_n in range(1, window_size + 1):
            my_neighbor = tokens[idx + a_n]
            if my_neighbor not in word2idx:
                word2idx[my_neighbor] = len(word2idx)
            cl.append(word2idx[my_neighbor])
        train_x_list.append(cl)
        if w not in word2idx:
            word2idx[w] = len(word2idx)
        train_y_list.append(word2idx[w])
train_X = np.array(train_x_list)
train_y_arr = np.array(train_y_list)

In [37]:
train_X

array([[0, 0, 1, 2],
       [0, 3, 2, 4],
       [3, 1, 4, 0],
       [1, 2, 0, 0],
       [0, 0, 4, 0],
       [0, 5, 0, 0],
       [0, 0, 6, 7],
       [0, 8, 7, 9],
       [8, 6, 9, 0],
       [6, 7, 0, 0]])

In [40]:
train_y_arr.reshape(-1)

array([3, 1, 2, 4, 5, 4, 8, 6, 7, 9])

In [38]:
train_y = keras.utils.to_categorical(train_y_arr)
train_y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

##  <font color='blue'>Task 2</font>: Implement the continuous bag of words model for estimating word embeddings

Implement the CBOW model for word embeddings: a CBOW with window size 2, which sums the input embeddings and from that hidden representations predicts the target token. 

You will:
* learn how to use the Kears `Embedding` layer 
* learn how to implement a FNN for learning embeddings with CBOW which *sums* the context embedding vectors (hint: `Lambda` layer and the keras backend function to `sum` over the dimensions)
* train the model for a few epochs using stochastic gradient descent (SGD)
* read off the learned embeddings $W$, store them in a gensim-readable file and inspect them

Here is our illustration of the model:


<img src="pics/cbow-model.png">
Another helpful illustration by Lilian Weng:<img src="pics/word2vec-cbow-matrices.png"> 


In [2]:
## your code here
embeddings_size = 64
vocab_size = None 


# print(model.summary())

In [4]:
# compile and train the model


##  <font color='blue'>Task 3</font>: Save & inspect the learned word embeddings


Now that you have trained the model:
    
* Extract the parameters $W$ (from the Embeddings layer)
* Save the parameters in a *word embeddings file* `embeds.out` (plained text file):
  - the first line defines the size of the embeddings: `num_words emb_size` (the number of words in the vocabularly and the defined embeddings size)
  - every line that follows contains the word followed by the numbers of each word embedding
* Show the first 5 lines of the file you just created 
* Load the vectors and show a few examples of closest words
* Think at how you can improve the model. What impacts the accuracy of the model? And the speed of training? What are ways to gauge the quality of such a model?

Example of embeddings learned from `tiny_corpus` (numbers can be different from yours):

```
> head -3 embeds.out
10 64
<pad> 0.014830918 0.017114433 -0.017562436 0.033561528 0.00035962294 0.004118913 -0.040838756 -0.03667029 -0.024223952 -0.028906876 -0.020306548 0.030102678 -0.0148919765 0.054747507 -0.020435825 -0.005233913 0.025159845 0.027897257 0.0036762147 0.014451144 0.036329776 -0.047892746 0.008516105 -0.015489587 -0.047726918 -0.01277938 -0.02608966 0.032392174 0.017537527 -0.04239431 0.0054095862 -0.038078584 -0.021500364 -0.003463915 0.0031234617 0.03047131 -0.01163403 -0.016979374 0.034163676 -0.044504337 -0.010839249 -0.02106181 -0.029292949 0.032988563 0.0068428325 0.014400309 -0.047024347 -0.016157033 0.042541306 -0.046763856 0.048780836 -0.019843327 -0.05658511 -0.030092832 -0.04061347 0.0031857535 0.0131299775 0.006951913 0.022310575 0.0017449302 0.027144292 0.011394488 -0.014438842 -0.05031742
is -0.012765145 0.0490117 -0.00768796 0.047519173 -0.0036907457 -0.023193887 -0.040275328 -0.034992553 0.02658589 -0.016711224 -0.025987871 0.011234962 -0.043054905 -0.019875554 0.035295177 0.010965967 -0.007958623 -0.0071568545 -0.012113648 -6.8368536e-05 0.02545237 0.03210017 -0.038777396 -0.021549145 -0.044163585 -0.008108679 -0.043482352 0.0041064247 0.01552576 -0.02667667 0.038252316 -0.019550333 0.030160375 0.010919128 0.009131351 -0.0076739364 0.030216074 -0.026353769 -0.028318474 0.043413714 -0.0033674275 -0.021961428 0.0134627 0.04476438 -0.03023459 0.01365161 0.0056271753 -0.006620986 0.0061316555 -0.012534681 0.030381938 0.021054488 -0.0072180103 0.0051654475 -0.038355142 -0.026047137 0.04695531 0.027473068 0.015156664 -0.0026225052 -0.017547552 0.044545818 -0.017625937 0.041487534
an -0.02792497 0.03405397 0.0009088284 -0.0070121214 -0.043711208 -0.043063454 0.040551104 0.015364666 0.026981615 -0.0020577777 -0.012685952 0.048457082 0.00990498 0.04459648 -0.046676215 0.0013320972 -0.008267676 0.013541599 0.025934678 0.004498395 -0.024405824 -0.048518687 0.01793391 -0.016555281 -0.0371649 -0.04145178 -0.020793486 0.047445916 0.02823504 0.015718877 0.042822 -0.043549605 -0.002715739 0.044998057 0.008055434 0.042783275 0.009758085 0.0059833084 -0.013179079 -0.05030296 -0.028446099 0.0151455905 0.037347842 -0.039894074 -0.020511352 -0.0028449045 -0.044684745 0.006346093 0.0054957103 -0.026536262 -0.042296298 -0.007664533 0.017584546 -0.043533787 -0.04807234 0.00088805275 -0.039341416 -0.009988039 5.4005533e-05 -0.016457608 0.0420327 0.043120813 0.01757149 0.010890282
this 0.0415787 -0.0086027505 -0.019305788 -0.017269727 -0.022438718 0.0006679795 0.0474798 0.014843477 0.008405418 0.02204414 -0.01822584 -0.020373298 -0.037552454 -0.0311565 0.007283304 0.04412359 -0.043263838 0.030828144 -0.0028054996 -0.025442088 -0.02153741 0.019842442 0.028329441 -0.019021718 -0.026186557 0.017510308 0.03005149 -0.01957564 -0.032455396 0.010885277 -0.010459727 -0.021340068 0.030479599 0.037034743 -0.030953838 -0.029407807 -0.04933766 -0.0063963323 -0.024726335 -0.04746379 -0.006584158 0.028048716 -0.018464241 0.025286743 -0.02888071 -0.035506 -0.024977053 -0.036528368 0.048054762 0.03790155 0.021394772 -0.019223671 -0.028742488 -0.047823258 0.029595247 0.023360644 -0.04435721 -0.037480954 -0.017783323 -0.04289337 -0.022487829 -0.0464871 0.045942437 -0.024913125
```

In [11]:
### your code here - extract the embedding parameters



In [12]:
## save the embeddings to file


In [19]:
## show the first 5 lines


10 64
<pad> 0.014830918 0.017114433 -0.017562436 0.033561528 0.00035962294 0.004118913 -0.040838756 -0.03667029 -0.024223952 -0.028906876 -0.020306548 0.030102678 -0.0148919765 0.054747507 -0.020435825 -0.005233913 0.025159845 0.027897257 0.0036762147 0.014451144 0.036329776 -0.047892746 0.008516105 -0.015489587 -0.047726918 -0.01277938 -0.02608966 0.032392174 0.017537527 -0.04239431 0.0054095862 -0.038078584 -0.021500364 -0.003463915 0.0031234617 0.03047131 -0.01163403 -0.016979374 0.034163676 -0.044504337 -0.010839249 -0.02106181 -0.029292949 0.032988563 0.0068428325 0.014400309 -0.047024347 -0.016157033 0.042541306 -0.046763856 0.048780836 -0.019843327 -0.05658511 -0.030092832 -0.04061347 0.0031857535 0.0131299775 0.006951913 0.022310575 0.0017449302 0.027144292 0.011394488 -0.014438842 -0.05031742
is -0.012765145 0.0490117 -0.00768796 0.047519173 -0.0036907457 -0.023193887 -0.040275328 -0.034992553 0.02658589 -0.016711224 -0.025987871 0.011234962 -0.043054905 -0.019875554 0.0352951

In [17]:
## load the vectors and show a few examples of closest words
## your code here


loading finished
